环境清单

numpy 

pandas

cpca    提取中文地址的库

jieba   分词工具

gensim   文本处理库

In [1]:
#pip install folium 可视化需要的库，可以不安装
import numpy as np
import pandas as pd
import time
import cpca 
from cpca import drawer
import jieba
from gensim import corpora, models, similarities
import warnings
warnings.filterwarnings("ignore")
 

In [2]:
def read_data(path):
    #导入数据,并对数据做一些处理
    df = pd.read_excel("{}.xlsx".format(path), dtype={'公司代码': 'str'})
    addr_df = df[["公司代码", "公司简称", "注册地址"]]
    addr_df["注册地址"]= addr_df["注册地址"].apply(lambda x: str(x).strip())
    return addr_df

In [3]:
def get_dataset(addr_df):
    '''
    #对地址做一个标准化处理,其中导入cpca的包进行处理
    考虑到了区为空和区不为空两种情况
    '''
    start = time.time()
    
    location_str = []
    for i in addr_df['注册地址']:
        location_str.append(i.strip())
        
    addr_cp = cpca.transform(location_str)
    
    #给结果表拼接唯一识别代码
    e_data = addr_df[["公司代码", "公司简称"]]
    addr_cpca = pd.concat([e_data, addr_cp], axis=1)
    
    #1.区不为空
    addr_cpca_1 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']!= '')]
    addr_cpca_1= addr_cpca_1.dropna()
    
    addr_cpca_11= addr_cpca_1[(addr_cpca['地址']!='')]
    addr_cpca_12= addr_cpca_11. dropna(subset=['地址'])
    
    #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
    addr_cpca_12['省市区'] = addr_cpca_12['省'] + addr_cpca_12['市'] + addr_cpca_12['区']
    
    addr_cpca_12['省市区长度']=addr_cpca_12['省市区'].apply(lambda x: len(x))
    count_1 = addr_cpca_12['省市区'].value_counts().reset_index()
    count_1= count_1.rename(columns={'index':'省市区', '省市区':'个数'})
    
    count_delete_1= count_1[count_1['个数']==1]
    dataset_1 = pd.merge(addr_cpca_12, count_delete_1, on = '省市区', how = 'left')
    dataset_1= dataset_1[dataset_1['个数']!=1]
    
    #2.区为空  
    addr_cpca_2 = addr_cpca[(addr_cpca['省']!= '')&(addr_cpca['市']!= '') & (addr_cpca['区']== '')]
    addr_cpca_2 = addr_cpca_2.dropna()
    
    addr_cpca_21= addr_cpca_2[(addr_cpca['地址']!='')]
    addr_cpca_22= addr_cpca_21. dropna(subset=['地址'])
    
    #将前三个字段完全拼接在一起进行分组然后组内进行相似度分析
    addr_cpca_22['省市区'] = addr_cpca_22['省'] + addr_cpca_22['市']
    
    addr_cpca_22['省市区长度']=addr_cpca_22['省市区'].apply(lambda x: len(x))
    count_2 = addr_cpca_22['省市区'].value_counts().reset_index()
    count_2= count_2.rename(columns={'index':'省市区', '省市区':'个数'})
    
    count_delete_2 = count_2[count_2['个数']==1]
    dataset_2 = pd.merge(addr_cpca_22, count_delete_2, on = '省市区', how = 'left')
    dataset_2 = dataset_2[dataset_2['个数']!=1]
    
    
    print("Time used:", round(time.time()-start), "s")
    
    return dataset_1, dataset_2

In [4]:
def cal_similar(doc_goal, document, ssim = 0.1):
#def cal_similar(doc_goai, document):
    '''
    分词;计算文本相似度，这里使用TF-IDF模型，如果有必要也可以使用其他文本向量化模型
    doc_goal,短文本，目标文档
    document,多个文本，被比较的多个文档
    '''
    all_doc_list=[]
    for doc in document:
        doc= "".join(doc)
        doc_list=[word for word in jieba.cut(doc)]
        all_doc_list.append(doc_list)

    #目标文档
    doc_goal = "".join(doc_goal)
    doc_goal_list = [word for word in jieba.cut(doc_goal)]
    
    #被比较的多个文档
    dictionary = corpora.Dictionary(all_doc_list)  #先用dictionary方法获词袋
    corpus = [dictionary.doc2bow(doc) for doc in all_doc_list]  #使用doc2bow制作预料库
    
    #目标文档
    doc_goal_vec = dictionary.doc2bow(doc_goal_list)
    tfidf = models.TfidfModel(corpus)#使用TF-DF模型对料库建模
    index = similarities.SparseMatrixSimilarity(tfidf[corpus], num_features = len(dictionary.keys()))#对每个目标文档,分析测文档的相似度
    
    #开始比较
    sims = index[tfidf[doc_goal_vec]]
    #similary= sorted(enumerate(sims),key=lambda item: -item[1])#根据相似度排序
    
    addr_dict={"被比较地址": document, "相似度": list(sims)}
    similary = pd.DataFrame(addr_dict)
    similary["目标地址"] = doc_goal
    similary_data = similary[["目标地址", "被比较地址", "相似度"]]
    similary_data= similary_data[similary_data["相似度"]>=ssim]
    
    return similary_data


In [5]:
def cycle_first(single_data):
    
    single_value = single_data.loc[:,["公司代码","地址"]].values #提取地址和ID
    
    cycle_data = pd. DataFrame([])
    for key, value in enumerate(single_value):
        if key < len(single_data)-1:
            doc_goal=list(value)[1:]
            document=list(single_data["地址"])[key+1:]
            cycle = cal_similar(doc_goal, document, ssim=0)
            cycle['目标地址代码'] = list(single_data["公司代码"])[key]
            cycle['目标地址省市区'] = list(single_data["省市区"])[key]
            cycle['被比较地址代码'] = list(single_data["公司代码"])[key+1:]
            cycle['被比较地址省市区'] = list(single_data["省市区"])[key+1:]
            cycle = cycle[["目标地址省市区","目标地址代码","目标地址","被比较地址省市区","被比较地址代码", "被比较地址", "相似度"]]
            #print("循环第",key,"个地址,得到表的行数,",len(cycle),",当前子循环计算进度,",key/len(cycle))
        cycle_data = cycle_data.append(cycle)
        cycle_data = cycle_data.drop_duplicates()
        cycle_data['adcode'] = single_data['adcode'].values[0]
   
    return cycle_data


def get_collect(dataset):
    '''
    计算同一区内地址的相似度，减少计算成本
    可以修改为计算同一个人的地址相似度
    '''
    start = time.time()
    
    #获取单个省市区的文档
    collect_data = pd.DataFrame([])
    ssq=list(set(dataset['省市区']))#如果要改成计算同一个人相似度，则这里改为身份证号
    for v, word in enumerate(ssq):
        single_data = dataset[dataset['省市区'] == word]
        print("循环第",v,"个省市区地址为:",word,",当前此区地址有:",len(single_data),",当前计算进度为:{:.1f}%" .format(v*100/len(ssq)))
        cycle_data = cycle_first(single_data)
        
        collect_data = collect_data.append(cycle_data)#将每个市区得到的结果放入一张表
        
        print("Time: %s" %time.ctime())
        print("-----------------------------------------------------------------------")
    
    print("Time used:",(time.time() - start), "s")
    
    return collect_data

In [6]:
df1 =read_data('创业板')
df2 = read_data('中小企业板')
df = pd.concat([df1,df2],axis=0,ignore_index=True)

In [7]:
df

,公司代码,公司简称,注册地址
0,300001,特锐德,山东省青岛市崂山区松岭路336号
1,300002,神州泰岳,北京市海淀区万泉庄路28号万柳新贵大厦A座6层601室
2,300003,乐普医疗,北京市昌平区超前路37号
3,300004,南风股份,广东省佛山市南海区小塘三环西路(狮南段)31号
4,300005,探路者,北京市昌平区科技园区永安路26号609
...,...,...,...
1768,002986,宇新股份,湖南省长沙市雨花区迎新路868号德思勤城市广场第A-2地块第7栋16层
1769,002987,京北方,北京市海淀区西三环北路25号7层
1770,002988,豪美新材,广东省清远市高新技术产业开发区泰基工业城
1771,002990,盛视科技,广东省深圳市福田区沙头街道天安社区泰然十路天安创新科技广场二期东座1601、1605


In [8]:
dataset_1, dataset_2  =get_dataset(df)
collect_data_1 = get_collect(dataset_1)
collect_data_1.sort_values(by=["相似度"], ascending=[False],inplace=True)

Building prefix dict from the default dictionary ...
Loading model from cache C:\Users\ADMINI~1\AppData\Local\Temp\jieba.cache


Time used: 0 s
循环第 0 个省市区地址为: 北京市市辖区门头沟区 ,当前此区地址有: 2 ,当前计算进度为:0.0%


Loading model cost 0.471 seconds.
Prefix dict has been built successfully.


Time: Wed Apr  6 15:49:42 2022
-----------------------------------------------------------------------
循环第 1 个省市区地址为: 广东省惠州市惠阳区 ,当前此区地址有: 2 ,当前计算进度为:0.5%
Time: Wed Apr  6 15:49:42 2022
-----------------------------------------------------------------------
循环第 2 个省市区地址为: 浙江省绍兴市柯桥区 ,当前此区地址有: 4 ,当前计算进度为:1.1%
Time: Wed Apr  6 15:49:42 2022
-----------------------------------------------------------------------
循环第 3 个省市区地址为: 福建省福州市鼓楼区 ,当前此区地址有: 7 ,当前计算进度为:1.6%
Time: Wed Apr  6 15:49:43 2022
-----------------------------------------------------------------------
循环第 4 个省市区地址为: 云南省昆明市官渡区 ,当前此区地址有: 2 ,当前计算进度为:2.2%
Time: Wed Apr  6 15:49:43 2022
-----------------------------------------------------------------------
循环第 5 个省市区地址为: 江苏省无锡市滨湖区 ,当前此区地址有: 6 ,当前计算进度为:2.7%
Time: Wed Apr  6 15:49:43 2022
-----------------------------------------------------------------------
循环第 6 个省市区地址为: 上海市市辖区宝山区 ,当前此区地址有: 2 ,当前计算进度为:3.3%
Time: Wed Apr  6 15:49:43 2022
---------------------------------------------

Time: Wed Apr  6 15:49:48 2022
-----------------------------------------------------------------------
循环第 56 个省市区地址为: 四川省成都市武侯区 ,当前此区地址有: 6 ,当前计算进度为:30.8%
Time: Wed Apr  6 15:49:48 2022
-----------------------------------------------------------------------
循环第 57 个省市区地址为: 安徽省宣城市宁国市 ,当前此区地址有: 4 ,当前计算进度为:31.3%
Time: Wed Apr  6 15:49:48 2022
-----------------------------------------------------------------------
循环第 58 个省市区地址为: 浙江省杭州市拱墅区 ,当前此区地址有: 2 ,当前计算进度为:31.9%
Time: Wed Apr  6 15:49:48 2022
-----------------------------------------------------------------------
循环第 59 个省市区地址为: 浙江省杭州市桐庐县 ,当前此区地址有: 2 ,当前计算进度为:32.4%
Time: Wed Apr  6 15:49:48 2022
-----------------------------------------------------------------------
循环第 60 个省市区地址为: 广东省佛山市禅城区 ,当前此区地址有: 3 ,当前计算进度为:33.0%
Time: Wed Apr  6 15:49:48 2022
-----------------------------------------------------------------------
循环第 61 个省市区地址为: 浙江省宁波市江北区 ,当前此区地址有: 2 ,当前计算进度为:33.5%
Time: Wed Apr  6 15:49:49 2022
---------------------------------

Time: Wed Apr  6 15:49:54 2022
-----------------------------------------------------------------------
循环第 112 个省市区地址为: 河南省焦作市中站区 ,当前此区地址有: 2 ,当前计算进度为:61.5%
Time: Wed Apr  6 15:49:54 2022
-----------------------------------------------------------------------
循环第 113 个省市区地址为: 新疆维吾尔自治区乌鲁木齐市新市区 ,当前此区地址有: 2 ,当前计算进度为:62.1%
Time: Wed Apr  6 15:49:54 2022
-----------------------------------------------------------------------
循环第 114 个省市区地址为: 江苏省无锡市惠山区 ,当前此区地址有: 2 ,当前计算进度为:62.6%
Time: Wed Apr  6 15:49:54 2022
-----------------------------------------------------------------------
循环第 115 个省市区地址为: 山东省潍坊市寿光市 ,当前此区地址有: 2 ,当前计算进度为:63.2%
Time: Wed Apr  6 15:49:54 2022
-----------------------------------------------------------------------
循环第 116 个省市区地址为: 广东省深圳市南山区 ,当前此区地址有: 84 ,当前计算进度为:63.7%
Time: Wed Apr  6 15:49:56 2022
-----------------------------------------------------------------------
循环第 117 个省市区地址为: 浙江省温州市永嘉县 ,当前此区地址有: 2 ,当前计算进度为:64.3%
Time: Wed Apr  6 15:49:56 2022
-------------------

Time: Wed Apr  6 15:50:02 2022
-----------------------------------------------------------------------
循环第 168 个省市区地址为: 山东省临沂市临沭县 ,当前此区地址有: 2 ,当前计算进度为:92.3%
Time: Wed Apr  6 15:50:02 2022
-----------------------------------------------------------------------
循环第 169 个省市区地址为: 江苏省苏州市吴江区 ,当前此区地址有: 8 ,当前计算进度为:92.9%
Time: Wed Apr  6 15:50:03 2022
-----------------------------------------------------------------------
循环第 170 个省市区地址为: 江苏省苏州市相城区 ,当前此区地址有: 3 ,当前计算进度为:93.4%
Time: Wed Apr  6 15:50:03 2022
-----------------------------------------------------------------------
循环第 171 个省市区地址为: 新疆维吾尔自治区昌吉回族自治州昌吉市 ,当前此区地址有: 2 ,当前计算进度为:94.0%
Time: Wed Apr  6 15:50:03 2022
-----------------------------------------------------------------------
循环第 172 个省市区地址为: 山东省淄博市沂源县 ,当前此区地址有: 2 ,当前计算进度为:94.5%
Time: Wed Apr  6 15:50:03 2022
-----------------------------------------------------------------------
循环第 173 个省市区地址为: 北京市市辖区房山区 ,当前此区地址有: 3 ,当前计算进度为:95.1%
Time: Wed Apr  6 15:50:03 2022
------------------

In [11]:
collect_data_1

,目标地址省市区,目标地址代码,目标地址,被比较地址省市区,被比较地址代码,被比较地址,相似度,adcode
2,浙江省绍兴市诸暨市,002203,店口镇工业区,浙江省绍兴市诸暨市,002590,店口镇工业区,1.000000,330681
1,安徽省宣城市宁国市,300475,经济技术开发区创业北路16号,安徽省宣城市宁国市,002538,经济技术开发区,1.000000,341881
6,广东省广州市天河区,300805,黄埔大道西平云路163号通讯大楼4层东部之三房,广东省广州市天河区,002967,黄埔大道西平云路163号,0.961010,440106
46,广东省深圳市南山区,300572,粤海街道学府路63号高新区联合总部大厦35楼,广东省深圳市南山区,002587,粤海街道学府路63号高新区联合总部大厦10楼,0.945883,440305
49,广东省深圳市南山区,300531,粤海街道学府路63号高新区联合总部大厦36楼,广东省深圳市南山区,002587,粤海街道学府路63号高新区联合总部大厦10楼,0.935560,440305
...,...,...,...,...,...,...,...,...
34,广东省深圳市南山区,300771,学苑大道1001号南山智园C3栋15层,广东省深圳市南山区,002660,松白路关外小白芒桑泰工业园,0.000000,440305
33,广东省深圳市南山区,300771,学苑大道1001号南山智园C3栋15层,广东省深圳市南山区,002618,高新园朗山一路丹邦科技大楼,0.000000,440305
10,广东省深圳市南山区,300771,学苑大道1001号南山智园C3栋15层,广东省深圳市南山区,002197,粤海街道西海岸大厦18楼,0.000000,440305
5,广东省深圳市南山区,300771,学苑大道1001号南山智园C3栋15层,广东省深圳市南山区,002139,高新技术产业园清华大学研究院B区413房,0.000000,440305


In [9]:
collect_data_1.to_csv('相似度排行表.csv')

In [10]:
#高相似度筛选之后进行地图可视化
collect_data = collect_data_1[collect_data_1["相似度"]>=0.8].sort_values(by=["相似度"], ascending=[False])
drawer.draw_locations(collect_data [cpca._ADCODE], "processed1.html")

In [13]:
df

,公司代码,公司简称,注册地址
0,300001,特锐德,山东省青岛市崂山区松岭路336号
1,300002,神州泰岳,北京市海淀区万泉庄路28号万柳新贵大厦A座6层601室
2,300003,乐普医疗,北京市昌平区超前路37号
3,300004,南风股份,广东省佛山市南海区小塘三环西路(狮南段)31号
4,300005,探路者,北京市昌平区科技园区永安路26号609
...,...,...,...
1768,002986,宇新股份,湖南省长沙市雨花区迎新路868号德思勤城市广场第A-2地块第7栋16层
1769,002987,京北方,北京市海淀区西三环北路25号7层
1770,002988,豪美新材,广东省清远市高新技术产业开发区泰基工业城
1771,002990,盛视科技,广东省深圳市福田区沙头街道天安社区泰然十路天安创新科技广场二期东座1601、1605


In [14]:
#节点表
df1 = pd.read_excel("创业板.xlsx", dtype={'公司代码': 'str'})
df2 = pd.read_excel("中小企业板.xlsx", dtype={'公司代码': 'str'})
df = pd.concat([df1,df2],axis=0,ignore_index=True)
#这里是给节点分配属性的过程，在信贷中则为客户的其他征信标签信息
df_node = df[['公司代码', '公司简称', '公司全称', '注册地址', '所属行业']]
df_node = df_node.rename(columns = {"公司代码": ":ID"})
df_node.to_csv("node.csv", index =False)


In [16]:
#边关系表
df_rela = collect_data[["目标地址代码", "被比较地址代码", "相似度"]]
df_relation = df_rela.rename(columns = {"目标地址代码": ":START_ID", "被比较地址代码": ":END_ID", "相似度":":TYPE"})
df_relation.to_csv("relationship.csv", index = False)